In [1]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.7/120.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [2]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/Amazon reviews/sample_data_for_topic_modelling.csv')

In [6]:
data.shape

(25129, 23)

In [7]:
data.head()

,Unnamed: 0.1,Unnamed: 0,ID,Subject,Description,sub_words_count,sub_char_count,desc_words_count,desc_char_count,Cleaned_Desc,...,desc_score,desc_sentiment,Date/Time Opened,Case Category,Category,Sub Category,Case Close Reason,Product Type,Origin,Top Management Flag
0,30,30,31,Planet App Feedback,Very poor customer service no response from yo...,3,19,21,143,Very poor customer service no response from yo...,...,0.9392,negative,2022-12-31T19:12:39.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,Two Wheeler,Mobile app,No
1,131,131,132,L&T Financial Services - General Enquiry,Two wheeler loan still not update\n\n-- \nDisc...,6,40,176,1115,Two wheeler loan still not update,...,0.6240,negative,2022-12-31T19:49:36.000+0000,Query,Payment & Refund,Payment Confirmation,Refer to this case id :17768143,Two Wheeler,Email,No
2,167,167,168,Loan not clear,NAME - NARMADA DAS\nMOBILE NO - 9678183958\nLO...,3,14,205,1265,NAME NARMADA DAS MOBILE NO LOAN NO C I have pa...,...,0.6760,negative,2022-12-31T20:07:44.000+0000,Query,Account Closure,Account Status is Active,"Dear Customer,\r\n\r\nThank you for writing to...",Consumer Loan,Email,No
3,226,226,227,L&T Financial Services - General Enquiry,"Dear Team,\n\n\nI have already paid the emi of...",6,40,214,1355,Dear Team I have already paid the emi of rd Ja...,...,0.4921,negative,2022-12-31T20:51:17.000+0000,Query,General Enquiry,LTFS Related,We acknowledge the receipt of your email and a...,Others,Email,No
4,292,292,293,LAN C022122501011708,"Dear Sir/Madam,\n\nI have taken loan from you ...",2,20,258,1571,Dear Sir Madam I have taken loan from you but ...,...,0.7081,negative,2022-12-31T21:58:37.000+0000,Query,Insurance,Insurance Related,We regret to inform you that we are unable to ...,Consumer Loan,Email,No


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25129 entries, 0 to 25128
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0.1              25129 non-null  int64  
 1   Unnamed: 0                25129 non-null  int64  
 2   ID                        25129 non-null  int64  
 3   Subject                   24794 non-null  object 
 4   Description               25129 non-null  object 
 5   sub_words_count           25129 non-null  int64  
 6   sub_char_count            25129 non-null  int64  
 7   desc_words_count          25129 non-null  int64  
 8   desc_char_count           25129 non-null  int64  
 9   Cleaned_Desc              25129 non-null  object 
 10  cleaned_desc_words_count  25129 non-null  int64  
 11  subject_score             25129 non-null  float64
 12  subject_sentiment         25129 non-null  object 
 13  desc_score                25129 non-null  float64
 14  desc_s

In [9]:
import re, string

In [10]:
def preprocess(CleanText):
    CleanText = CleanText.str.replace("(<br/>)", "")
    CleanText = CleanText.str.replace('(<a).*(>).*(</a>)', '')
    CleanText = CleanText.str.replace('(&amp)', '')
    CleanText = CleanText.str.replace('(&gt)', '')
    CleanText = CleanText.str.replace('(&lt)', '')
    CleanText = CleanText.str.replace('(\xa0)', ' ')
    # CleanText = " ".str.join(CleanText.str.split())  
    return CleanText
data['Cleaned_Desc'] = preprocess(data['Cleaned_Desc'])

In [11]:
mask = (data['desc_score']>=0.8)
data_mod = data[mask]

In [12]:
data_mod.head()

,Unnamed: 0.1,Unnamed: 0,ID,Subject,Description,sub_words_count,sub_char_count,desc_words_count,desc_char_count,Cleaned_Desc,...,desc_score,desc_sentiment,Date/Time Opened,Case Category,Category,Sub Category,Case Close Reason,Product Type,Origin,Top Management Flag
0,30,30,31,Planet App Feedback,Very poor customer service no response from yo...,3,19,21,143,Very poor customer service no response from yo...,...,0.9392,negative,2022-12-31T19:12:39.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,Two Wheeler,Mobile app,No
9,533,533,534,Planet App Feedback,"Services, Unsuccessful Mandate registration",3,19,4,43,Services Unsuccessful Mandate registration,...,0.8449,negative,2023-01-01T02:55:14.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,MFI,Mobile app,No
11,576,576,577,Planet App Feedback,"Services, Unsuccessful Mandate registration, N...",3,19,5,53,Services Unsuccessful Mandate registration Non...,...,0.8663,negative,2023-01-01T03:20:58.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,Two Wheeler,Mobile app,No
12,590,590,591,Planet App Feedback,"Technical, Slowness in Application, No option ...",3,19,13,88,Technical Slowness in Application No option is...,...,0.8383,negative,2023-01-01T03:34:53.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,NaN,Mobile app,No
20,738,738,739,Planet App Feedback,"Services, Unsuccessful Mandate registration, b...",3,19,11,83,Services Unsuccessful Mandate registration bad...,...,0.9320,negative,2023-01-01T04:05:21.000+0000,Feedback,PLANET feedback,Negative Feedback,NaN,NaN,Mobile app,No


In [13]:
data_mod.shape

(4388, 23)

In [14]:
type(data['Cleaned_Desc'])

pandas.core.series.Series

In [15]:
# create model 
 
model = BERTopic(verbose=True)
 
#convert to list 
docs = data_mod.Cleaned_Desc.to_list()
 
topics, probabilities = model.fit_transform(docs)

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

2023-05-23 09:01:08,249 - BERTopic - Transformed documents to Embeddings
2023-05-23 09:01:55,939 - BERTopic - Reduced dimensionality
2023-05-23 09:01:56,271 - BERTopic - Clustered reduced embeddings


In [20]:
model.get_topic_freq()

,Topic,Count
0,0,825
1,1,538
2,2,347
3,3,323
4,-1,279
5,4,233
6,5,135
7,6,112
8,7,110
9,8,95


In [17]:
model.get_topic_info()

,Topic,Count,Name
0,-1,279,-1_to_my_the_this
1,0,825,0_drop_end_response_no
2,1,538,1_slowness_application_technical_in
3,2,347,2_mandate_registration_unsuccessful_services
4,3,323,3_enquiryunsuccessful_dropped_callcall_nutan
5,4,233,4_barrier_agentlanguage_transferred_another
6,5,135,5_services_bad_service_very
7,6,112,6_breaking_request_back_voice
8,7,110,7_loan_my_not_apply
9,8,95,8_emi_my_account_amount


In [18]:
def output(x):
  return model.get_topic(x)

In [22]:
for j in range(54):
  print("\033[1m"+'Topic', j, 'words:'+"\033[0m", end='  ')
  for k in range(10):
    print(output(j)[k][0], end=' ')
  print('                                                                                      ')

Topic 0 words:  drop end response no call customer dropped form server from                                                                                       
Topic 1 words:  slowness application technical in slow very working app too services                                                                                       
Topic 2 words:  mandate registration unsuccessful services personal service aisa blank white dot                                                                                       
Topic 3 words:  enquiryunsuccessful dropped callcall nutan geeting viche sub categorycall cenquiryunsuccessful droppedf                                                                                       
Topic 4 words:  barrier agentlanguage transferred another enquiryunsuccessful callcall to marathi agent discount                                                                                       
Topic 5 words:  services bad service very good worst poor not care manners

In [25]:
print(len(model.get_topic(28)))

10


In [26]:
model.visualize_barchart()

In [27]:
model.visualize_topics()

In [28]:
# model.visualize_heatmap()

In [29]:
model1 = BERTopic(nr_topics=10)

In [30]:
docs = data_mod.Cleaned_Desc.to_list()
 
topics, probabilities = model1.fit_transform(docs)

2023-05-23 09:04:35,422 - BERTopic - Transformed documents to Embeddings
2023-05-23 09:04:57,351 - BERTopic - Reduced dimensionality
2023-05-23 09:04:57,531 - BERTopic - Clustered reduced embeddings
2023-05-23 09:04:59,878 - BERTopic - Reduced number of topics from 78 to 10


In [31]:
model1.get_topic_freq()

,Topic,Count
0,0,2091
1,1,644
2,2,644
3,-1,369
4,3,307
5,4,162
6,5,81
7,6,47
8,7,31
9,8,12


In [32]:
def output_1(x):
  return model1.get_topic(x)

In [33]:
for j in range(9):
  print("\033[1m"+'Topic', j, 'words:'+"\033[0m", end='  ')
  for k in range(10):
    print(output_1(j)[k][0], end=' ')
  print('                                                                                      ')

Topic 0 words:  call customer the response no and end drop to is                                                                                       
Topic 1 words:  callcall enquiryunsuccessful barrier agentlanguage transferred dropped another to unsuccessful cx                                                                                       
Topic 2 words:  application slowness technical in app slow working very not properly                                                                                       
Topic 3 words:  registration mandate unsuccessful services login register very app unable slow                                                                                       
Topic 4 words:  services bad service very good worst not technical poor your                                                                                       
Topic 5 words:  surendra unsuccessful dropped callcall brack voice issues coming surrender server                                

In [34]:
model1.visualize_barchart()

In [35]:
model1.visualize_topics()